In [21]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import seaborn as sns 


In [22]:
df = pd.read_csv("diamonds_train.csv", index_col=0)

In [23]:
df["carat"] = 10*df["carat"]

df["cut"] = df["cut"].apply(lambda x: x.replace("Fair","0"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Very Good","2"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Good","1"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Ideal","3"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Premium","4"))
df["cut"] = df["cut"].astype(int)

df["color"] = df["color"].apply(lambda x: x.replace("D","10"))
df["color"] = df["color"].apply(lambda x: x.replace("E","9"))
df["color"] = df["color"].apply(lambda x: x.replace("F","8"))
df["color"] = df["color"].apply(lambda x: x.replace("G","6"))
df["color"] = df["color"].apply(lambda x: x.replace("H","5"))
df["color"] = df["color"].apply(lambda x: x.replace("I","4"))
df["color"] = df["color"].apply(lambda x: x.replace("J","0"))
df["color"] = df["color"].astype(int)

df["clarity"] = df["clarity"].apply(lambda x: x.replace("SI1","3"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("SI2","2"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("I1","0"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("IF","10"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VVS1","9"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VVS2","8"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VS1","7"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VS2","6"))
df["clarity"] = df["clarity"].astype(int)


In [24]:
df[df["price"] > 1800].sort_values(by="price", ascending=False).tail(20)

,carat,cut,color,clarity,depth,table,x,y,z,price
9391,7.0,1,8,2,63.6,56.0,5.63,5.56,3.56,1806
34382,5.0,2,9,7,60.0,60.0,5.13,5.20,3.10,1806
34262,10.0,0,4,0,68.6,56.0,6.20,6.08,4.22,1805
20036,6.1,2,8,6,62.6,60.0,5.34,5.39,3.36,1805
20364,5.5,3,8,7,61.4,55.0,5.24,5.28,3.23,1805
35783,5.4,3,6,7,61.8,55.3,5.22,5.24,3.23,1804
29166,5.2,3,6,7,61.9,55.5,5.13,5.16,3.18,1803
14227,5.3,3,8,6,61.6,57.0,5.21,5.15,3.19,1803
6900,7.0,0,5,3,65.0,59.0,5.54,5.50,3.59,1803
30575,5.2,3,6,7,61.2,56.0,5.19,5.23,3.19,1803


In [25]:
# df.drop(df[df["carat"] > 28].index, inplace=True)

In [26]:
df[df["price"] < 800].sort_values(by="price", ascending=False).tail(20)

,carat,cut,color,clarity,depth,table,x,y,z,price
11382,2.3,2,6,8,60.4,58.0,3.97,4.01,2.41,354
23211,3.1,2,0,3,59.4,62.0,4.39,4.43,2.62,353
16123,2.3,2,5,7,61.0,57.0,3.94,3.96,2.41,353
29449,2.3,2,9,6,63.8,55.0,3.85,3.92,2.48,352
9318,3.0,1,4,2,63.3,56.0,4.26,4.30,2.71,351
1162,3.0,1,0,3,63.8,56.0,4.23,4.26,2.71,351
18471,3.0,1,0,3,63.4,54.0,4.23,4.29,2.70,351
35842,3.0,3,4,2,62.0,54.0,4.31,4.34,2.68,348
39266,2.0,4,9,2,60.2,62.0,3.79,3.75,2.27,345
4504,3.2,4,9,0,60.9,58.0,4.38,4.42,2.68,345


In [27]:
X = np.array(df[["carat","color","cut","clarity" ,"y"]])
y = np.array(df["price"])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.0001, random_state = 235)

In [29]:
svr_rbf = SVR(kernel='rbf', C=10000, gamma="auto", epsilon=0.1, max_iter=999999)
svr_rbf.fit(X_train, y_train)

C:\Users\xyang\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=999999).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=10000, gamma='auto', max_iter=999999)

In [30]:
y_test_pred = svr_rbf.predict(X_test)
print(r2_score(y_test, y_test_pred))
np.sqrt(mean_squared_error(y_test, y_test_pred))

0.9943447946667261


83.34133930737806

In [31]:
# C = 10000/ 555.1367153891555

# x --> 550.8611897991423
# y --> 541.3904810385991
# z --> 546.4043006671172

In [32]:
pd.DataFrame({"y_test":y_test,"y_pred": y_test_pred})

,y_test,y_pred
0,2041,1892.885690
1,640,742.255395
2,3583,3538.344228
3,616,603.497763
4,1243,1229.419286


### PREDECIR TEST

In [33]:
X_pred = pd.read_csv("diamonds_test.csv", index_col = 0)

In [34]:
X_pred["carat"] = 10*X_pred["carat"]

In [35]:
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Fair","0"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Very Good","2"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Good","1"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Ideal","3"))
X_pred["cut"] = X_pred["cut"].apply(lambda x: x.replace("Premium","4"))
X_pred["cut"] = X_pred["cut"].astype(int)

In [36]:


X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("D","10"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("E","9"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("F","8"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("G","6"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("H","5"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("I","4"))
X_pred["color"] = X_pred["color"].apply(lambda x: x.replace("J","0"))
X_pred["color"] = X_pred["color"].astype(int)

X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("SI1","3"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("SI2","2"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("I1","0"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("IF","10"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VVS1","9"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VVS2","8"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VS1","7"))
X_pred["clarity"] = X_pred["clarity"].apply(lambda x: x.replace("VS2","6"))
X_pred["clarity"] = X_pred["clarity"].astype(int)


In [37]:
X_pred = np.array(X_pred[["carat","cut", "color", "clarity", "y"]])

In [38]:
pd.DataFrame(X_pred)

,0,1,2,3,4
0,3.0,3.0,5.0,2.0,4.43
1,3.4,3.0,10.0,10.0,4.46
2,15.7,2.0,4.0,6.0,7.55
3,3.1,3.0,5.0,6.0,4.36
4,15.1,1.0,4.0,9.0,7.21
...,...,...,...,...,...
13444,10.5,3.0,8.0,2.0,6.68
13445,2.7,2.0,6.0,7.0,4.16
13446,15.0,4.0,10.0,6.0,7.37
13447,17.0,4.0,8.0,2.0,7.68


In [39]:
predictions_submit = svr_rbf.predict(X_pred)
predictions_submit

array([2368.24043098, 7158.19104086, 8001.8036501 , ..., 7160.5117069 ,
       7207.61825522, 7155.62797437])

In [40]:
pd.DataFrame(predictions_submit)

,0
0,2368.240431
1,7158.191041
2,8001.803650
3,1956.053119
4,9756.320893
...,...
13444,6944.069742
13445,4399.481956
13446,7160.511707
13447,7207.618255


In [41]:
submission = pd.DataFrame({"id": range(len(predictions_submit)), "price": predictions_submit})

In [42]:
submission.to_csv("submission6.csv", index = False)